In [62]:
import os
import json
from warnings import warn
import pandas as pd
import numpy as np
from tqdm import tqdm

import sys
sys.path.append("../src/optimisation/src/")
from connections import DB

from prepare_data import OBJECT_ID_COL

pd.set_option('display.max_columns', None)
tqdm.pandas()

In [2]:
GIS_FILEPATH = '/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/Сведения_об_объектах_жилищного_фонда_на_23-10-2022/prepared_gis_house_data.pickle'
DMR_FILEPATH = '/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/address_register/prepared_addr_register.pickle'
prepared_gis_house_data = pd.read_pickle(GIS_FILEPATH).drop(columns=OBJECT_ID_COL)
prepared_dmr = pd.read_pickle(DMR_FILEPATH)

In [3]:
prepared_gis_house_data.head(2)

,ADDRESS_GIS,ADDRESS_GIS_CODE,GUID_FIAS,OKTMO_CODE,MANAGEMENT_METHOD,MANAGEMENT_OGRN,MANAGEMENT_KPP,MANAGEMENT_NAME,HOUSE_TYPE,CONDITION,TOTAL_AREA,LIVING_AREA,DEMOLITION_DATE,CADASTRAL_NUM,GUID_HOUSE
0,"108809, Москва г, п. Марушкинское, д. Марушкин...",f93a19c3-ffab-48b5-92c3-d10a49166a72,F93A19C3-FFAB-48B5-92C3-D10A49166A72,45949000,Не выбран,,,NaN,Жилой,Исправный,396.2,NaN,NaN,50:26:0170402:2106,dc41973a-8cdd-4298-b7d7-365cd5b96a09
1,"108809, Москва г, п. Марушкинское, д. Марушкин...",9e595cc4-fa15-4d22-bc17-380b691f47c1,9E595CC4-FA15-4D22-BC17-380B691F47C1,45949000,Не выбран,,,NaN,Жилой,Исправный,124.9,NaN,NaN,нет,407142e2-e27c-4973-b614-7cd939e5a037


In [4]:
gis_cols = prepared_gis_house_data.columns

In [5]:
prepared_dmr.head(2)

,OBJECT_TYPE,ON_MOSCOW_TERRITORY,ADDRESS,SIMPLE_ADDRESS,STREET,LOCAL_OBJECT_TYPE,LOCAL_OBJECT_NUM,KORPUS_NUM,STROENIE_NUM,ADM_AREA,DISTRICT,NUM_ADDR_REGISTER,DATE_ADDR_REGISTER,GUID_FIAS,DATE_FIAS,KAD_N,KAD_ZU,KLADR_CODE,ADDR_STATUS,GEODATA,LAT,LON,OBJECT_ID
0,Здание,да,"Российская Федерация, город Москва, внутригоро...","Косинская улица, дом 26А",Косинская улица,дом,26А,NaN,NaN,Восточный административный округ,муниципальный округ Вешняки,3303053,03.08.2004,235212A3-01E8-4CC3-87D5-59F00C83898A,27.02.2012,77:03:0007004:1064,77:03:0007004:6443,77000000000040000,Внесён в ГКН,"[[37.8279504545784, 55.7176609928454], [37.828...",55.717477,37.828208,d2000323020834c81f704195dd68b59499d4800f4b2cae...
1,Здание,да,"Российская Федерация, город Москва, внутригоро...","Гороховский переулок, дом 21",Гороховский переулок,дом,21,NaN,NaN,Центральный административный округ,муниципальный округ Басманный,1011856,13.07.2005,533D296D-1ECC-49EA-9156-DCFA8E38E4D8,27.02.2012,77:01:0003010:1018,77:01:0003010:4146,77000000000112200,Внесён в ГКН,"[[37.6682995208392, 55.7662431483298], [37.668...",55.766233,37.668037,5d6cf51cd08bca0baf6680258a2e6a3a293b264b64d6c4...


In [6]:
dmr_cols = prepared_dmr.columns

In [7]:
dmr_fias = set(prepared_dmr['GUID_FIAS'].dropna())
dmr_kadn = set(prepared_dmr['KAD_N'].dropna())
dmr_kadzu = set(prepared_dmr['KAD_ZU'].dropna())

In [8]:
houses_data_fias = set(prepared_gis_house_data['GUID_FIAS'].dropna())
houses_data_kadn = set(prepared_gis_house_data['CADASTRAL_NUM'].dropna())

In [9]:
print('fias:', len(houses_data_fias), len(dmr_fias))
print('kadn', len(houses_data_kadn), len(dmr_kadn))
print('kadn', len(houses_data_kadn), len(dmr_kadzu))

fias: 68159 165900
kadn 47093 126254
kadn 47093 53071


In [10]:
print(len(houses_data_fias & dmr_fias))
print(len(houses_data_kadn & dmr_kadn))
print(len(houses_data_kadn & dmr_kadzu))

38764
31245
6


In [11]:
data_merged_fias = prepared_dmr.merge(
    prepared_gis_house_data,
    on=['GUID_FIAS'],
    how='left'
    )

In [12]:
data_merged_kadn = prepared_dmr.dropna(subset=['KAD_N']).merge(
    prepared_gis_house_data.dropna(subset=['CADASTRAL_NUM']),
    left_on=['KAD_N'],
    right_on=['CADASTRAL_NUM'],
    how='inner'
    )

In [13]:
data_merged_kadzu = prepared_dmr.dropna(subset=['KAD_ZU']).merge(
    prepared_gis_house_data.dropna(subset=['CADASTRAL_NUM']),
    left_on=['KAD_ZU'],
    right_on=['CADASTRAL_NUM'],
    how='inner'
    )

In [52]:
data_merged = pd.concat([data_merged_fias, data_merged_kadn, data_merged_kadzu])

data_merged.loc[data_merged['GUID_FIAS_x'].notna(),'GUID_FIAS'] = data_merged.loc[data_merged['GUID_FIAS_x'].notna(), 'GUID_FIAS_x']

data_merged['has_house_data'] = data_merged['GUID_HOUSE'].notna()
data_merged = data_merged.drop(columns=['GUID_FIAS_x', 'GUID_FIAS_y'])
data_merged = data_merged.sort_values(by='has_house_data')
print(data_merged.shape)
data_merged = data_merged.drop_duplicates(subset=OBJECT_ID_COL, keep='last')
data_merged = data_merged.drop(columns='has_house_data')
data_merged = data_merged.reset_index(drop=True)
print(data_merged.shape)
print(prepared_dmr.shape)

(205716, 38)
(174145, 37)
(174145, 23)


In [48]:
final_gis = data_merged[list(gis_cols) + [OBJECT_ID_COL]].copy()
final_gis = final_gis.dropna(
    subset=[col for col in final_gis.columns if col not in [OBJECT_ID_COL, 'GUID_FIAS']
    ], how='all')
final_gis

,ADDRESS_GIS,ADDRESS_GIS_CODE,GUID_FIAS,OKTMO_CODE,MANAGEMENT_METHOD,MANAGEMENT_OGRN,MANAGEMENT_KPP,MANAGEMENT_NAME,HOUSE_TYPE,CONDITION,TOTAL_AREA,LIVING_AREA,DEMOLITION_DATE,CADASTRAL_NUM,GUID_HOUSE,OBJECT_ID
135086,"115054, Москва г, ул. Бахрушина, д. 10, строен...",ffbc94d2-7371-426a-9aaa-2f606fd1be86,FFBC94D2-7371-426A-9AAA-2F606FD1BE86,45376000,УО,5137746116646,770501001,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,2184.3,2099.5,NaN,NaN,22e4df6f-c568-4b7d-b119-cdb90deddf1c,a1b50bfc376ea37edebd16ddd131b106bf8d8c846983a9...
135087,"121357, Москва г, ул. Артамонова, д. 8, корп. 2",0da29a7f-e80f-415a-8c73-4c780014ea98,0DA29A7F-E80F-415A-8C73-4C780014EA98,45329000,УО,1157746682610,773101001,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,3187.0,NaN,NaN,NaN,769ca40a-b4a8-4991-9a51-42117cfddd44,9a11acf2996873fc758e8f2454f47250bd5239153c8dc8...
135088,"125466, Москва г, ул. Соколово-Мещерская, д. 2...",3ad6b6a9-b08b-46f1-ac84-3656647678e9,101367BD-9F16-4CB1-AB95-1D29F046B45E,45366000,УО,,,NaN,Жилой,NaN,NaN,NaN,NaN,нет,507ec941-0ae9-4b7b-a421-4a720b32f5fa,5c2c5f1e51c57e6fa9d5a0e71c2e0c3adbece48dee122d...
135089,"125466, Москва г, ул. Соколово-Мещерская, д. 2...",20fa36fb-95c7-42db-9eb8-2bc7a6710cef,00FD3737-F8B7-4832-A0C7-51EC21572264,45366000,УО,,,NaN,Жилой дом блокированной застройки,Исправный,2544.4,NaN,NaN,нет,10c28fe0-d012-47dc-942b-dc486a2e6a56,4c4a73fa11c8817c1726cd5e0762088ddfa0db1e92032f...
135090,"125466, Москва г, ул. Соколово-Мещерская, д. 2...",5125b410-3e83-44e1-89fa-3e3b2359d240,5125B410-3E83-44E1-89FA-3E3B2359D240,45366000,УО,,,NaN,Жилой дом блокированной застройки,Исправный,508.4,NaN,NaN,нет,6aa41721-81ad-48d3-b6fe-f89cf4f6c168,1899f036c5adf9e920a84e42d0e2b95d3d45f52a9a1899...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174140,"115477, Москва г, ул. Кантемировская, д. 12, к...",0ca5b4b7-fc96-4178-9668-3f2c49f6155b,0CA5B4B7-FC96-4178-9668-3F2C49F6155B,45917000,ЖК,1037739400490,772401001,"ЖИЛИЩНО-СТРОИТЕЛЬНЫЙ КООПЕРАТИВ ""МОСКВОРЕЧЬЕ-14""",Многоквартирный,Исправный,11231.5,11180.8,NaN,77:05:0005010:1007,90fae13e-fd53-4a1f-89f0-547556dcc231,34fc7aadc034fd82429cb35c1af5c7c66bc4f2d971d0fb...
174141,"121165, Москва г, ул. Киевская, д. 24",bfb7b541-5302-406e-bc63-c6266c5d3019,40858C85-90C9-4EB1-8E6F-6A3C93EC7278,45318000,УО,1137746560148,773001001,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,15744.0,NaN,NaN,нет,c09e5384-44c7-4fcd-96a6-6a7477b3fcfb,2515a3f3d4ae2cf2711a94a83be2362a11e9c2e3395f55...
174142,"105215, Москва г, ул. 5-я Парковая, д. 64, кор...",7fd82f59-9a9b-4df5-91b9-ed86d3252788,7FD82F59-9A9B-4DF5-91B9-ED86D3252788,45313000,ЖК,1037739647857,771901001,"ЖИЛИЩНО-СТРОИТЕЛЬНЫЙ КООПЕРАТИВ ""ИЗМАЙЛОВСКИЙ""",Многоквартирный,Исправный,4342.9,3595.1,NaN,77:03:0005010:1032,b98f0153-fb0a-4bbd-9943-8e5fa1b7ee44,ad4219c23696441d9b2d9e2011d78f995a1e30288bd7de...
174143,"123103, Москва г, ул. Таманская, д. 36, корп. 10",bb467bdb-a57b-4cbe-97cf-bbe55198793c,BB467BDB-A57B-4CBE-97CF-BBE55198793C,45371000,Не выбран,,,NaN,Жилой,Исправный,627.1,NaN,NaN,нет,fbf3b169-4241-45a7-98d1-eb807891cd2d,0bb34143d5d99920cea1e47ddb5404c55f7dc1e3f6d4a7...


In [53]:
final_dmr = data_merged[dmr_cols].copy()
final_dmr

,OBJECT_TYPE,ON_MOSCOW_TERRITORY,ADDRESS,SIMPLE_ADDRESS,STREET,LOCAL_OBJECT_TYPE,LOCAL_OBJECT_NUM,KORPUS_NUM,STROENIE_NUM,ADM_AREA,DISTRICT,NUM_ADDR_REGISTER,DATE_ADDR_REGISTER,GUID_FIAS,DATE_FIAS,KAD_N,KAD_ZU,KLADR_CODE,ADDR_STATUS,GEODATA,LAT,LON,OBJECT_ID
0,Здание,да,"Российская Федерация, город Москва, внутригоро...","Косинская улица, дом 26А",Косинская улица,дом,26А,NaN,NaN,Восточный административный округ,муниципальный округ Вешняки,3303053,03.08.2004,235212A3-01E8-4CC3-87D5-59F00C83898A,27.02.2012,77:03:0007004:1064,77:03:0007004:6443,77000000000040000,Внесён в ГКН,"[[37.8279504545784, 55.7176609928454], [37.828...",55.717477,37.828208,d2000323020834c81f704195dd68b59499d4800f4b2cae...
1,Здание,да,"город Москва, 5-я Чоботовская аллея, дом 19, с...","5-я Чоботовская аллея, дом 19, строение Б/Н",5-я Чоботовская аллея,дом,19,NaN,Б/Н,Западный административный округ,муниципальный округ Ново-Переделкино,,NaN,NaN,NaN,77:07:0015005:2038,NaN,77000000000312000,Внесён в ГКН,"[[37.3614058876712, 55.6518353537253], [37.361...",55.651861,37.361526,74000c2eee2bdf4aa615cfde7630839d96f053316ac4fb...
2,Здание,да,"город Москва, 1-я Чоботовская аллея, дом 22А, ...","1-я Чоботовская аллея, дом 22А, строение Б/Н",1-я Чоботовская аллея,дом,22А,NaN,Б/Н,Западный административный округ,муниципальный округ Ново-Переделкино,0,NaN,NaN,NaN,77:00:0000000:41731:77:07:0015005:1966,NaN,77000000000715300,Внесён в ГКН,"[[37.3505728672034, 55.6536960121684], [37.350...",55.653725,37.350604,a0135cd4bcf784802eab90706286e3d965343c8f95f034...
3,Здание,да,"город Москва, проспект Мира, дом 186А, строение 9","проспект Мира, дом 186А, строение 9",проспект Мира,дом,186А,NaN,9,Северо-Восточный административный округ,муниципальный округ Алексеевский,,NaN,C7D52B05-2B90-4240-89BF-3E8A6FDC6385,23.05.2013,NaN,77:02:0019010:0014,77000000000185600,NaN,"[[37.6515906049217, 55.8281606690497], [37.651...",55.828119,37.651592,31a3c58cb2be8df7782b60a308a4d33c177b99af5c073e...
4,Здание,да,"город Москва, улица Щепкина, дом 42, строение Б/Н","улица Щепкина, дом 42, строение Б/Н",улица Щепкина,дом,42,NaN,Б/Н,Центральный административный округ,муниципальный округ Мещанский,,NaN,NaN,NaN,NaN,NaN,77000000000317100,NaN,"[[37.6300624595472, 55.7825126085941], [37.630...",55.782479,37.630075,76249b1db911d5ae4113750ec9ff00762ded7f78dc48eb...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174140,Здание,да,"Российская Федерация, город Москва, внутригоро...","Кантемировская улица, дом 12, корпус 2",Кантемировская улица,дом,12,2,NaN,Южный административный округ,муниципальный округ Москворечье-Сабурово,5010244,20.10.2004,0CA5B4B7-FC96-4178-9668-3F2C49F6155B,27.02.2012,77:05:0005010:1007,NaN,77000000000049200,Внесён в ГКН,"[[37.6706948541158, 55.6381058726656], [37.670...",55.638106,37.670111,34fc7aadc034fd82429cb35c1af5c7c66bc4f2d971d0fb...
174141,Здание,да,"Российская Федерация, город Москва, внутригоро...","Киевская улица, дом 24",Киевская улица,дом,24,NaN,NaN,Западный административный округ,муниципальный округ Дорогомилово,7102585,20.02.2003,40858C85-90C9-4EB1-8E6F-6A3C93EC7278,27.02.2012,77:07:0007002:1063,77:07:0007002:54,77000000000146400,Внесён в ГКН,"[[37.5390492124944, 55.7378222495242], [37.539...",55.737726,37.540747,2515a3f3d4ae2cf2711a94a83be2362a11e9c2e3395f55...
174142,Здание,да,"Российская Федерация, город Москва, внутригоро...","5-я Парковая улица, дом 64, корпус 4",5-я Парковая улица,дом,64,4,NaN,Восточный административный округ,муниципальный округ Северное Измайлово,3201533,03.11.2003,7FD82F59-9A9B-4DF5-91B9-ED86D3252788,27.02.2012,77:03:0005010:1032,77:03:0005010:3,77000000000221800,Внесён в ГКН,"[[37.7900148452773, 55.8092078445284], [37.790...",55.808862,37.790137,ad4219c23696441d9b2d9e2011d78f995a1e30288bd7de...
174143,Здание,да,"город Москва, Таманская улица, дом 36, корпус 10","Таманская улица, дом 36, корпус 10",Таманская улица,дом,36,10,NaN,Северо-Западный административный округ,муниципальный округ Хорошево-Мневники,8000

In [58]:
final_houses = data_merged.dropna(subset=['LAT', 'LON', 'TOTAL_AREA'])
final_houses['OBJECT_TYPE'] = 'многоквартирный дом'

/var/folders/7q/8vvk22jj58jd3fvwfg98269m0000gn/T/ipykernel_31338/187087627.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_houses['OBJECT_TYPE'] = 'многоквартирный дом'


In [59]:
final_houses

,OBJECT_TYPE,ON_MOSCOW_TERRITORY,ADDRESS,SIMPLE_ADDRESS,STREET,LOCAL_OBJECT_TYPE,LOCAL_OBJECT_NUM,KORPUS_NUM,STROENIE_NUM,ADM_AREA,DISTRICT,NUM_ADDR_REGISTER,DATE_ADDR_REGISTER,GUID_FIAS,DATE_FIAS,KAD_N,KAD_ZU,KLADR_CODE,ADDR_STATUS,GEODATA,LAT,LON,OBJECT_ID,ADDRESS_GIS,ADDRESS_GIS_CODE,OKTMO_CODE,MANAGEMENT_METHOD,MANAGEMENT_OGRN,MANAGEMENT_KPP,MANAGEMENT_NAME,HOUSE_TYPE,CONDITION,TOTAL_AREA,LIVING_AREA,DEMOLITION_DATE,CADASTRAL_NUM,GUID_HOUSE
135086,многоквартирный дом,да,"Российская Федерация, город Москва, внутригоро...","улица Бахрушина, дом 10, строение 3",улица Бахрушина,дом,10,NaN,3,Центральный административный округ,муниципальный округ Замоскворечье,1001314,24.05.2002,FFBC94D2-7371-426A-9AAA-2F606FD1BE86,24.11.2011,77:01:0002012:1017,NaN,77000000000081800,Внесён в ГКН,"[[37.6347869776043, 55.735330239525], [37.6345...",55.735592,37.634462,a1b50bfc376ea37edebd16ddd131b106bf8d8c846983a9...,"115054, Москва г, ул. Бахрушина, д. 10, строен...",ffbc94d2-7371-426a-9aaa-2f606fd1be86,45376000,УО,5137746116646,770501001,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,2184.3,2099.5,NaN,NaN,22e4df6f-c568-4b7d-b119-cdb90deddf1c
135087,многоквартирный дом,да,"Российская Федерация, город Москва, внутригоро...","улица Артамонова, дом 8, корпус 2",улица Артамонова,дом,8,2,NaN,Западный административный округ,муниципальный округ Фили-Давыдково,7103698,24.12.2003,0DA29A7F-E80F-415A-8C73-4C780014EA98,27.02.2012,77:07:0009003:1020,NaN,77000000000076700,Внесён в ГКН,"[[37.4572289466062, 55.720043356036], [37.4568...",55.720403,37.457119,9a11acf2996873fc758e8f2454f47250bd5239153c8dc8...,"121357, Москва г, ул. Артамонова, д. 8, корп. 2",0da29a7f-e80f-415a-8c73-4c780014ea98,45329000,УО,1157746682610,773101001,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,3187.0,NaN,NaN,NaN,769ca40a-b4a8-4991-9a51-42117cfddd44
135089,многоквартирный дом,да,"город Москва, Соколово-Мещерская улица, дом 29...","Соколово-Мещерская улица, дом 29, корпус 32",Соколово-Мещерская улица,дом,29,32,NaN,Северо-Западный административный округ,муниципальный округ Куркино,8008952,09.06.2007,00FD3737-F8B7-4832-A0C7-51EC21572264,27.02.2012,77:08:0001005:1057,77:08:0001007:1,77000000000452000,Внесён в ГКН,"[[37.3784480515434, 55.9064309844859], [37.378...",55.906276,37.378763,4c4a73fa11c8817c1726cd5e0762088ddfa0db1e92032f...,"125466, Москва г, ул. Соколово-Мещерская, д. 2...",20fa36fb-95c7-42db-9eb8-2bc7a6710cef,45366000,УО,,,NaN,Жилой дом блокированной застройки,Исправный,2544.4,NaN,NaN,нет,10c28fe0-d012-47dc-942b-dc486a2e6a56
135090,многоквартирный дом,да,"Российская Федерация, город Москва, внутригоро...","Соколово-Мещерская улица, дом 29, корпус 9",Соколово-Мещерская улица,дом,29,9,NaN,Северо-Западный административный округ,муниципальный округ Куркино,8009639,15.01.2010,5125B410-3E83-44E1-89FA-3E3B2359D240,27.02.2012,77:08:0001005:1049,77:08:0001005:3122,77000000000452000,Внесён в ГКН,"[[37.3803489398641, 55.906909054396], [37.3804...",55.906924,37.380579,1899f036c5adf9e920a84e42d0e2b95d3d45f52a9a1899...,"125466, Москва г, ул. Соколово-Мещерская, д. 2...",5125b410-3e83-44e1-89fa-3e3b2359d240,45366000,УО,,,NaN,Жилой дом блокированной застройки,Исправный,508.4,NaN,NaN,нет,6aa41721-81ad-48d3-b6fe-f89cf4f6c168
135091,многоквартирный дом,да,"Российская Федерация, город Москва, внутригоро...","4-й Верхний Михайловский проезд, дом 10, корпус 3",4-й Верхний Михайловский проезд,дом,10,3,NaN,Южный административный округ,муниципальный округ Донской,5004313,24.12.2002,20749B8B-FACB-4529-BC0C-268887FB8968,27.02.2012,77:05:0001011:1042,NaN,77000000000187200,Внесён в ГКН,"[[37.6016559165121, 55.7067489262928], [37.601...",55.706783,37.602183,ecd4be6fbdf3567b224e3aa38d8f639dd0378a3e0d9021...,"115419, Москва г, проезд. 4-й Верхний Михайлов...",20749b8b-facb-4529-bc0c-268887fb8968,45915000,УО,1147746013524,772601001,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,3848.4,3363.8,NaN,NaN,9321e463-7f68-4abc-89ed-

In [64]:
CONFIG_PATH = '/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/db_config.json'
with open(CONFIG_PATH, mode='r') as db_file:
    db_config = json.load(db_file)

databaase = DB(db_config)

In [70]:
# databaase.load_to_bd(final_dmr, 'all_dmr_houses_data')
# databaase.load_to_bd(final_gis, 'all_gis_houses_data')

In [71]:
databaase.load_to_bd(final_houses, 'apartment_houses_all_data')

In [72]:
databaase.get_table_from_bd('apartment_houses_all_data')

Connection to PostgreSQL DB successful


../src/optimisation/src/connections.py:56: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f'select * from {table_name}', connection)


,OBJECT_TYPE,ON_MOSCOW_TERRITORY,ADDRESS,SIMPLE_ADDRESS,STREET,LOCAL_OBJECT_TYPE,LOCAL_OBJECT_NUM,KORPUS_NUM,STROENIE_NUM,ADM_AREA,DISTRICT,NUM_ADDR_REGISTER,DATE_ADDR_REGISTER,GUID_FIAS,DATE_FIAS,KAD_N,KAD_ZU,KLADR_CODE,ADDR_STATUS,GEODATA,LAT,LON,OBJECT_ID,ADDRESS_GIS,ADDRESS_GIS_CODE,OKTMO_CODE,MANAGEMENT_METHOD,MANAGEMENT_OGRN,MANAGEMENT_KPP,MANAGEMENT_NAME,HOUSE_TYPE,CONDITION,TOTAL_AREA,LIVING_AREA,DEMOLITION_DATE,CADASTRAL_NUM,GUID_HOUSE
0,многоквартирный дом,да,"Российская Федерация, город Москва, внутригоро...","улица Бахрушина, дом 10, строение 3",улица Бахрушина,дом,10,None,3,Центральный административный округ,муниципальный округ Замоскворечье,1001314,24.05.2002,FFBC94D2-7371-426A-9AAA-2F606FD1BE86,24.11.2011,77:01:0002012:1017,None,77000000000081800,Внесён в ГКН,"{{37.6347869776043,55.735330239525},{37.634592...",55.735592,37.634462,a1b50bfc376ea37edebd16ddd131b106bf8d8c846983a9...,"115054, Москва г, ул. Бахрушина, д. 10, строен...",ffbc94d2-7371-426a-9aaa-2f606fd1be86,45376000,УО,5137746116646,770501001,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,2184.3,2099.5,None,None,22e4df6f-c568-4b7d-b119-cdb90deddf1c
1,многоквартирный дом,да,"Российская Федерация, город Москва, внутригоро...","улица Артамонова, дом 8, корпус 2",улица Артамонова,дом,8,2,None,Западный административный округ,муниципальный округ Фили-Давыдково,7103698,24.12.2003,0DA29A7F-E80F-415A-8C73-4C780014EA98,27.02.2012,77:07:0009003:1020,None,77000000000076700,Внесён в ГКН,"{{37.4572289466062,55.720043356036},{37.456852...",55.720403,37.457119,9a11acf2996873fc758e8f2454f47250bd5239153c8dc8...,"121357, Москва г, ул. Артамонова, д. 8, корп. 2",0da29a7f-e80f-415a-8c73-4c780014ea98,45329000,УО,1157746682610,773101001,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,3187.0,NaN,None,None,769ca40a-b4a8-4991-9a51-42117cfddd44
2,многоквартирный дом,да,"город Москва, Соколово-Мещерская улица, дом 29...","Соколово-Мещерская улица, дом 29, корпус 32",Соколово-Мещерская улица,дом,29,32,None,Северо-Западный административный округ,муниципальный округ Куркино,8008952,09.06.2007,00FD3737-F8B7-4832-A0C7-51EC21572264,27.02.2012,77:08:0001005:1057,77:08:0001007:1,77000000000452000,Внесён в ГКН,"{{37.3784480515434,55.9064309844859},{37.37861...",55.906276,37.378763,4c4a73fa11c8817c1726cd5e0762088ddfa0db1e92032f...,"125466, Москва г, ул. Соколово-Мещерская, д. 2...",20fa36fb-95c7-42db-9eb8-2bc7a6710cef,45366000,УО,,,None,Жилой дом блокированной застройки,Исправный,2544.4,NaN,None,нет,10c28fe0-d012-47dc-942b-dc486a2e6a56
3,многоквартирный дом,да,"Российская Федерация, город Москва, внутригоро...","Соколово-Мещерская улица, дом 29, корпус 9",Соколово-Мещерская улица,дом,29,9,None,Северо-Западный административный округ,муниципальный округ Куркино,8009639,15.01.2010,5125B410-3E83-44E1-89FA-3E3B2359D240,27.02.2012,77:08:0001005:1049,77:08:0001005:3122,77000000000452000,Внесён в ГКН,"{{37.3803489398641,55.906909054396},{37.380442...",55.906924,37.380579,1899f036c5adf9e920a84e42d0e2b95d3d45f52a9a1899...,"125466, Москва г, ул. Соколово-Мещерская, д. 2...",5125b410-3e83-44e1-89fa-3e3b2359d240,45366000,УО,,,None,Жилой дом блокированной застройки,Исправный,508.4,NaN,None,нет,6aa41721-81ad-48d3-b6fe-f89cf4f6c168
4,многоквартирный дом,да,"Российская Федерация, город Москва, внутригоро...","4-й Верхний Михайловский проезд, дом 10, корпус 3",4-й Верхний Михайловский проезд,дом,10,3,None,Южный административный округ,муниципальный округ Донской,5004313,24.12.2002,20749B8B-FACB-4529-BC0C-268887FB8968,27.02.2012,77:05:0001011:1042,None,77000000000187200,Внесён в ГКН,"{{37.6016559165121,55.7067489262928},{37.60164...",55.706783,37.602183,ecd4be6fbdf3567b224e3aa38d8f639dd0378a3e0d9021...,"115419, Москва г, проезд. 4-й Верхний Михайлов...",20749b8b-facb-4529-bc0c-268887fb8968,45915000,УО,1147746013524,772601001,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,3848.4,3363.8,None,None,9321e463-7f68-4abc-89ed-005feb4